# Solución etapa 3 - FE - V3

In [122]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl

In [123]:
# Cargar los datasets
import pandas as pd

historicos_ordenes_cleaned = pd.read_csv('../data/cleaned/historicos_ordenes_cleaned.csv')
registros_condiciones_cleaned = pd.read_csv('../data/cleaned/registros_condiciones_cleaned.csv')
caracteristicas_equipos_cleaned = pd.read_csv('../data/cleaned/caracteristicas_equipos_cleaned.csv')

## Feature eng

### Target

In [124]:
# Generar target

target_column="Fallo"

def code_target(val):
    if val == 'Correctivo':
        return 1
    else:
        return 0


historicos_ordenes_cleaned[target_column] = historicos_ordenes_cleaned["Tipo_Mantenimiento"].apply(code_target)
historicos_ordenes_cleaned.drop("Tipo_Mantenimiento", axis=1, inplace=True)

### historicos_ordenes

In [125]:
tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0
 1           2          244  2020-01-01 01:00:00                4274.63                35  Planta Norte        0
 2           3          297  2020-01-01 02:00:00                5752                   25  Planta Norte        0
 3           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1
 4           5          403  2020-01-01 04:00:00                7048.79                28  Planta Norte        1
(9951, 7)


In [126]:
historicos_ordenes_cleaned['Fecha'] = pd.to_datetime(historicos_ordenes_cleaned['Fecha'])

historicos_ordenes_cleaned['Año'] = historicos_ordenes_cleaned['Fecha'].dt.year
historicos_ordenes_cleaned['Mes'] = historicos_ordenes_cleaned['Fecha'].dt.month
historicos_ordenes_cleaned['Día'] = historicos_ordenes_cleaned['Fecha'].dt.day

tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo    Año    Mes    Día
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------  -----  -----  -----
 0           1           36  2020-01-01 00:00:00                6754.12                 3  Planta Norte        0   2020      1      1
 1           2          244  2020-01-01 01:00:00                4274.63                35  Planta Norte        0   2020      1      1
 2           3          297  2020-01-01 02:00:00                5752                   25  Planta Norte        0   2020      1      1
 3           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1   2020      1      1
 4           5          403  2020-01-01 04:00:00                7048.79                28  Planta Norte        1   2020      1      1
(9951, 10)


> Compensamos el desajuste en la escala de 'Tiempo_medio_entre_fallos' para que el modelo entendia mejor las relaciones

In [127]:
# Tiempo medio entre fallos por mes

# Filtrar solo los registros con fallos
fallos = historicos_ordenes_cleaned[historicos_ordenes_cleaned['Fallo'] == 1].copy()

# Ordenar por fecha
fallos = fallos.sort_values(by=['ID_Equipo', 'Fecha'])

fallos['Diferencia_Horas'] = fallos.groupby('ID_Equipo')['Fecha'].diff().dt.total_seconds() / 3600 # Compensación de escala

promedio_horas_entre_fallos_por_mes = (
    fallos.groupby(['ID_Equipo', 'Año', 'Mes','Día'])['Diferencia_Horas']
          .mean()
          .reset_index(name='Promedio_Horas_Entre_Fallos_por_mes')
)

tabl(promedio_horas_entre_fallos_por_mes)

      ID_Equipo    Año    Mes    Día    Promedio_Horas_Entre_Fallos_por_mes
--  -----------  -----  -----  -----  -------------------------------------
 0            1   2020      2      4                                    nan
 1            1   2020      2     25                                    507
 2            1   2020      3     17                                    492
 3            1   2020      6     12                                   2100
 4            1   2020      7     26                                   1034
(4895, 5)


In [128]:
# Mezclar historicos_ordenes_cleaned + promedio_horas_entre_fallos_por_mes

historicos_promedio_horas_entre_fallos_por_mes = pd.merge(historicos_ordenes_cleaned, promedio_horas_entre_fallos_por_mes, on=['ID_Equipo','Año','Mes','Día'], how='inner')
tabl(historicos_promedio_horas_entre_fallos_por_mes)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion       Fallo    Año    Mes    Día    Promedio_Horas_Entre_Fallos_por_mes
--  ----------  -----------  -------------------  ---------------------  ----------------  ------------  -------  -----  -----  -----  -------------------------------------
 0           4          431  2020-01-01 03:00:00                5690.14                 8  Planta Sur          1   2020      1      1                                    nan
 1           5          403  2020-01-01 04:00:00                7048.79                28  Planta Norte        1   2020      1      1                                    nan
 2           6          477  2020-01-01 05:00:00                1519.65                40  Planta Norte        1   2020      1      1                                    nan
 3           7          323  2020-01-01 06:00:00                9148.27                37  Planta Sur          1   2020      1      1  

In [129]:
# Verificamos nulos
print(historicos_promedio_horas_entre_fallos_por_mes.isna().sum())
historicos_promedio_horas_entre_fallos_por_mes.dropna(inplace=True)

ID_Orden                                 0
ID_Equipo                                0
Fecha                                    0
Costo_Mantenimiento                      0
Duracion_Horas                           0
Ubicacion                                0
Fallo                                    0
Año                                      0
Mes                                      0
Día                                      0
Promedio_Horas_Entre_Fallos_por_mes    507
dtype: int64


In [130]:
# Eliminar columnas sin valor de historicos_promedio_horas_entre_fallos_por_mes
historicos_hefbm_clean = historicos_promedio_horas_entre_fallos_por_mes.drop(columns=['ID_Orden','Costo_Mantenimiento','Duracion_Horas','Fecha'])
tabl(historicos_hefbm_clean)

      ID_Equipo  Ubicacion       Fallo    Año    Mes    Día    Promedio_Horas_Entre_Fallos_por_mes
--  -----------  ------------  -------  -----  -----  -----  -------------------------------------
39          161  Planta Sur          1   2020      1      4                                     37
44          286  Planta Este         1   2020      1      4                                     83
47           49  Planta Norte        1   2020      1      5                                      5
49           49  Planta Sur          1   2020      1      5                                      5
68          185  Planta Este         1   2020      1      6                                     90
(4569, 7)


In [131]:
historicos_hefbm_clean.groupby(['ID_Equipo','Año','Mes']).count().loc[1]

Ubicacion  Fallo  Día  Promedio_Horas_Entre_Fallos_por_mes
Año  Mes                                                            
2020 2            1      1    1                                    1
     3            1      1    1                                    1
     6            1      1    1                                    1
     7            1      1    1                                    1
     8            1      1    1                                    1
     12           1      1    1                                    1
2021 1            1      1    1                                    1

> Coincidencia a nivel de mes entre historico y registro

### registros_condiciones

In [132]:
registros_condiciones_cleaned.drop(columns=['ID_Registro'], index=1, inplace=True)

registros_condiciones_cleaned['Fecha']=pd.to_datetime(registros_condiciones_cleaned['Fecha'])
registros_condiciones_cleaned['Año'] = registros_condiciones_cleaned['Fecha'].dt.year
registros_condiciones_cleaned['Mes'] = registros_condiciones_cleaned['Fecha'].dt.month
registros_condiciones_cleaned['Día'] = registros_condiciones_cleaned['Fecha'].dt.day

tabl(registros_condiciones_cleaned)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    Año    Mes    Día
--  -----------  -------------------  ---------------  ----------------  ------------------  -----  -----  -----
 0          260  2020-01-01 00:00:00           101.43              1.22               71849   2020      1      1
 2          443  2020-01-01 02:00:00            76.04              2.93               33106   2020      1      1
 3          281  2020-01-01 03:00:00            97.09              3.25               31744   2020      1      1
 4          427  2020-01-01 04:00:00           139.42              4.22               78104   2020      1      1
 5           48  2020-01-01 05:00:00            98.77              2.73               89664   2020      1      1
(8999, 8)


In [137]:
registros_condiciones_cleaned.groupby(['ID_Equipo','Año','Mes']).count().loc[1]

Fecha  Temperatura_C  Vibracion_mm_s  Horas_Operativas  Día
Año  Mes                                                             
2020 1        2              2               2                 2    2
     3        2              2               2                 2    2
     4        3              3               3                 3    3
     6        1              1               1                 1    1
     7        2              2               2                 2    2
     8        2              2               2                 2    2
     10       1              1               1                 1    1
     12       2              2               2                 2    2

> Compensamos el desajuste en la escala de 'Tiempo_medio_entre_fallos' para que el modelo entendia mejor las relaciones

In [ ]:
# Agrupar por 'ID_Equipo', 'Año' y 'Mes', y calcular los promedios

promedios_por_equipo_mes = (
    registros_condiciones_cleaned.groupby(['ID_Equipo', 'Año', 'Mes'])[['Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']]
      .mean()
      .reset_index()
)

# Renombrar las columnas para claridad
promedios_por_equipo_mes.rename(columns={
    'Temperatura_C': 'Promedio_Temperatura_C',
    'Vibracion_mm_s': 'Promedio_Vibracion_mm_s',
    'Horas_Operativas': 'Promedio_Horas_Operativas'
}, inplace=True)

promedios_por_equipo_mes['Promedio_Horas_Operativas']=promedios_por_equipo_mes['Promedio_Horas_Operativas']/3600 # Compensación de escala

In [ ]:
# Verificar y eliminar nulos
promedios_por_equipo_mes.isna().sum()
promedios_por_equipo_mes.dropna(inplace=True)
promedios_por_equipo_mes.shape

### Unir merged_hitorico_registro + promedios + caracteristicas

In [ ]:
# Unir datasets
final_data = pd.merge(historicos_hefbm_clean, promedios_por_equipo_mes, on=['ID_Equipo','Año','Mes'], how='inner')
final_data = pd.merge(final_data, caracteristicas_equipos_cleaned, on='ID_Equipo', how='inner')

final_data['Horas_Recomendadas_Revision']=final_data['Horas_Recomendadas_Revision']/3600 # Compensamos ajuste en escala

tabl(final_data)

In [ ]:
# Limpiar columnas accesorias
final_data.drop(columns=['ID_Equipo','Año', 'Mes', 'Día'], inplace=True)

tabl(final_data)

### Duplicados

In [ ]:
# Eliminar duplicados

print(headr('Duplicados final_data:'),final_data.duplicated().sum())
# final_data.drop_duplicates(inplace=True)
# tabl(final_data)

In [ ]:
# Verificamos nulos
print(headr('Nulos final_data:'),final_data.isna().sum())

In [ ]:
columnas_con_nulos = final_data.columns[final_data.isna().any()].tolist()
columnas_con_nulos

In [ ]:
# Imputamos nulos
from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy="median")

# final_data[columnas_con_nulos] = imputer.fit_transform(final_data[columnas_con_nulos])

In [ ]:
tabl(final_data)

In [ ]:
final_data.info()

### Volvemos a explorar con columnas codificadas


In [ ]:
# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(final_data)
categorical_columns = categorical_columns_selector(final_data)

print(headr("Numerical columns"), numerical_columns)
print(headr("Categorical columns"), categorical_columns)

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

final_data_coded =final_data.copy()
final_data_coded[categorical_columns] =final_data_coded[categorical_columns].apply(LabelEncoder().fit_transform)   
tabl(final_data_coded)

In [ ]:
# Evaluamos la aportación de cada columna
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2

X= final_data_coded.drop(target_column, axis=1)
y = final_data_coded[target_column]

fvalue_selector = SelectKBest(f_classif, k=2)

X_kbest = fvalue_selector.fit(X,y)

feature_scores = pd.DataFrame({"Feature": X.columns,"Score": X_kbest.scores_}).sort_values(by="Score", ascending=False)

print(headr("Feature scores"))
round(feature_scores,2)

In [ ]:
# Eliminamos columnas sin peso

columns_to_drop = feature_scores[feature_scores['Score'] == 0]['Feature']
final_data.drop(columns=columns_to_drop, inplace=True)
tabl(final_data)

## Exportar 

In [ ]:
final_data.to_csv('../data/preprocessed/preprocessed_data_v3.csv', index=False)